# Code used to generate the data set

In [26]:
import numpy as np
import os
import math
import shutil
# from moviepy.video.io.VideoFileClip import VideoFileClip
# from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

In [27]:
if not os.path.exists('videos'):
    shutil.unpack_archive('videos.zip', 'videos')


In [28]:
#parse metadata
with open("data_unprocessed/metadata.txt") as file:
    lines =  np.array([line.rstrip().split(";") for line in file])


In [29]:
newpath = r'./temp' 
if not os.path.exists(newpath):
    os.makedirs(newpath)

Ideally we would use a python library to download the youtube videos here. Unfortunatly a recent change to Youtubes cypher system has rendered all of these tools non-functional. Since our data set has a small number of long videos, we have opted to download them manually.

Currently the raw videos are not included in the repo, later I will include them in a zip archive if the github file limit allows it

In [30]:
from moviepy.config import FFMPEG_BINARY
import subprocess

def ffmpeg_extract_subclip(
    inputfile, start_time, end_time, outputfile=None, logger="bar"
):
    """Makes a new video file playing video file between two times.

    Parameters
    ----------

    inputfile : str
      Path to the file from which the subclip will be extracted.

    start_time : float
      Moment of the input clip that marks the start of the produced subclip.

    end_time : float
      Moment of the input clip that marks the end of the produced subclip.

    outputfile : str, optional
      Path to the output file. Defaults to
      ``<inputfile_name>SUB<start_time>_<end_time><ext>``.
    """
    if not outputfile:
        name, ext = os.path.splitext(inputfile)
        t1, t2 = [int(1000 * t) for t in [start_time, end_time]]
        outputfile = "%sSUB%d_%d%s" % (name, t1, t2, ext)

    cmd = [
        FFMPEG_BINARY,
        "-y",
        "-ss",
        "%0.2f" % start_time,
        "-i",
        inputfile,
        "-to",
        "%0.2f" % end_time,
        "-map",
        "0",
        "-vcodec",
        "copy",
        "-acodec",
        "copy",
        "-copyts",
        outputfile,
    ]
    subprocess.run(cmd)

In [31]:
links = np.unique(lines.transpose()[4])

i=0
while i < len(lines):
    
    if float(lines[i][5]) == float(lines[i+1][5])  and lines[i][4] == lines[i+1][4]:
        sample = lines[i if lines[i][6]> lines[i+1][6]  else i+1]
        start = float(lines[i][5])
        end = max(float(lines[i][6]), float(lines[i+1][6]))
        
        name = lines[i][0]+"&"+lines[i+1][0]
        i+=2
    else:
        sample = lines[i]
        start = float(sample[5])
        end = float(sample[6])
        name = sample[0]
        i+=1

    idx = np.where(links == sample[4])
    video = sample[4].split("=")[-1]
    ## If its the first video, remove once all have been downloaded
    # if idx[0][0] == 0:
    ffmpeg_extract_subclip("videos/"+video+".mp4", start, end, "cropped/" + name+".mp4")
        
       



Create a zip archive of the output so the videos can be commited to github

In [32]:
if os.path.isfile("videos_processed.zip"):
    os.remove("videos_processed.zip")
shutil.make_archive("videos_processed", 'zip', "cropped")

'c:\\Users\\mikeG\\Documents\\school\\cisc-452\\CISC-452-Group-22\\videos_processed.zip'

In [44]:
# cropped\1_2018-07-12_1.mp4

run = "34_2018-07-06_2"

import cv2
vidcap = cv2.VideoCapture('cropped\\'+run+'.mp4')
success,image = vidcap.read()
count = 0

with open("data_unprocessed/skeletons/"+run+".data") as file:
    data =  np.array([line.rstrip().split("#") for line in file])

frame_count =int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT)) - 1

print(data)

offset = frame_count - int(data[-1][0])

print ("Number of frames: ", int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT)) - 1)
while success:
  if count < offset:
      count += 1
      success,image = vidcap.read()
      continue
  cv2.imwrite("img/frame%d.jpg" % (count-offset), image)     # save frame as JPEG file      
  success,image = vidcap.read()
  print('Read a new frame: ', success)
  count += 1

[['1'
  '640.7,787.7;674.7,689.5;795.5,708.4;746.4,723.5;663.3,674.4;644.5,791.4;769.0,719.7;704.9,561.2;704.9,557.4;697.3,481.9;663.3,432.8;716.2,485.6;742.6,546.0;659.6,572.5;584.1,572.5;550.1,512.1']
 ['2'
  '644.6,786.8;674.6,689.1;798.6,707.9;746.0,722.9;663.4,674.1;644.6,790.6;768.6,719.2;704.7,561.3;704.7,557.6;697.2,482.4;659.6,433.6;716.0,486.2;746.0,546.3;659.6,572.6;584.4,572.6;546.9,508.7']
 ['3'
  '635.5,786.8;660.3,687.4;774.0,705.1;717.2,719.3;642.6,669.6;635.5,790.4;745.6,708.7;695.9,552.3;695.9,548.8;695.9,470.6;660.3,431.6;713.6,477.7;734.9,523.9;642.6,563.0;571.5,577.2;546.6,516.8']
 ['4'
  '648.2,784.1;714.0,853.6;739.7,696.2;684.8,699.9;615.2,670.6;629.8,787.7;706.7,696.2;681.1,538.8;681.1,538.8;688.4,458.3;661.0,438.2;710.4,476.6;732.3,524.2;633.5,546.2;560.3,579.1;549.3,516.9']
 ['5'
  '625.3,782.9;614.9,696.0;712.2,678.6;667.0,696.0;590.6,668.2;618.4,782.9;684.4,682.1;667.0,522.3;667.0,518.8;670.5,442.4;663.5,431.9;722.6,477.1;722.6,518.8;618.4,529.2;548.9,581.4